In [32]:
import pickle

# shape: activations[layer name][batch index] = nparray of (image idx, channel-mean activation)
with open("./outputs/imagenet-activations.pickle", "rb") as f:
    activations = pickle.load(f)

# Compute channel-wise mean and stddev

In [44]:
import numpy as np

layer = "fc"

channel_means = np.zeros((1000,))
channel_stds = np.zeros((1000,))

acts = np.zeros((1281167, 1000))
i = 0
for image_idx in activations[layer]:
    acts[image_idx] = activations[layer][image_idx]

channel_means = np.mean(acts, axis=0)
channel_stds = np.std(acts, axis=0)

# Find highest-activating dataset examples for every neuron

In [22]:
import torchvision
import torch
# Reproduce order from all_of_imagenet.py
imagenet = torchvision.datasets.ImageNet('/datasets/imagenet/', split="train")
torch.manual_seed(1)
dataloader = torch.utils.data.DataLoader(imagenet, batch_size=64,
                                         shuffle=True)

In [14]:
print(len(imagenet.samples))

1281167


In [49]:
from shutil import copyfile
import os

for neuron_idx in range(1000):
    top_dset_indices = np.argsort(-acts[:, neuron_idx])[:10]
    
#     break
    
    os.makedirs(f"./outputs/imagenet-activation-analysis/{str(neuron_idx)}/", exist_ok=True)
    
    for top_idx, image_idx in enumerate(top_dset_indices):
        filename, _ = imagenet.samples[image_idx]
        copyfile(filename, f"./outputs/imagenet-activation-analysis/{str(neuron_idx)}/{str(top_idx)}.jpg")
        